In [ ]:
import logging
from pathlib import Path

import cartopy.feature as cfeature
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.gridspec as gridspec
import cmocean.cm as cmo
from matplotlib.patches import Rectangle
import numpy as np

import matplotlib.pyplot as plt
import xarray as xr


In [ ]:
base_path = Path("/work/n01/n01/fwg/dwbc-proj")
ctd_path = base_path / "data/raw/ctd_data/cchdo_search_results-1/7_3175MB91_ctd.nc"

In [ ]:
base_path = Path('/work/n01/n01/fwg/dwbc-proj')
raw_path = base_path / 'data/raw'
processed_path = base_path / 'data/processed'
run_suff = ""
figure_path = base_path / "figures"


from matplotlib import rc

SMALL_SIZE = 8
MEDIUM_SIZE = 8
BIGGER_SIZE = 8
rc('font',**{'family':'sans-serif','sans-serif':['Arial']})
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
text_width = 5.5  # in inches

cm = 1/2.54
dpi = 300

In [ ]:
ds = xr.open_dataset(ctd_path)
ds = ds.where(((ds["latitude"] >= -10) * (ds["latitude"] <= 0)
               * (ds["longitude"] >= -50) * (ds["longitude"] <= - 30)), drop=True)

In [ ]:
# Open the datasats
ds_bathymetry = xr.open_dataset(raw_path / 'GEBCO-bathymetry-data/gebco_2021_n30.0_s-30.0_w-85.0_e-10.0.nc')
ds_bathymetry = ds_bathymetry.coarsen(lon=5, lat=5, boundary='trim').mean()
ds_climatological_gamma_n = xr.open_dataset(processed_path / 'climatological_gamman.nc', decode_times=False)

# Set up the canvas
pad = 35
fig = plt.figure(figsize=(text_width, 3))#9 * cm))

gs = gridspec.GridSpec(2, 2,
                        width_ratios=[1, 1],
                        height_ratios=[1, 1/16]
                        )

ax1 = fig.add_subplot(gs[0, 0], projection=ccrs.PlateCarree())

# Plot the bathymetry
cax_bathy = ax1.pcolormesh(ds_bathymetry['lon'],
                            ds_bathymetry['lat'],
                            -ds_bathymetry['elevation'],
                            shading='nearest',
                            rasterized=True,
                            cmap=cmo.deep,
                            vmin=0
                            )

# Add some land
ax1.add_feature(cfeature.NaturalEarthFeature('physical',
                                                'land',
                                                '110m',
                                                edgecolor='face',
                                                facecolor='grey'
                                            ))

y0 = ds_climatological_gamma_n['lat'].min()
ywid = ds_climatological_gamma_n['lat'].max() - y0
x0 = ds_climatological_gamma_n['lon'].min()
xwid = ds_climatological_gamma_n['lon'].max() - x0
ax1.add_patch(Rectangle((x0, y0), xwid, ywid, ec='red', fc='none'))

# Axes limits, labels and features
ax1.axhline(0, c='k', ls='--')

ax1.set_ylim(-12, 30)
ax1.set_xlim(-85, -25)

ax1.set_xticks(np.arange(-85, -24, 10), crs=ccrs.PlateCarree())
ax1.set_yticks(np.arange(-10, 31, 10), crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter()
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)


ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
ax1.set_title('The Tropical Atlantic')
ax1.set_title('(a)', loc='left')

# Colorbars
cbax1 = fig.add_subplot(gs[1, 0])
cb1 = plt.colorbar(cax_bathy, cax=cbax1, label='Depth (m)', orientation='horizontal')

# Initial condition plots
ax2 = fig.add_subplot(gs[:, 1])


ax2.set_title('Temperature staircase')
ax2.set_title('(b)', loc='left')
ax2.set_xlabel('In-situ temperature ($^\\circ$C)', usetex=True)
ax2.set_ylabel('Pressure (dBar)')

cmo.tempo_r.set_bad('grey')

prof = 0
ax2.plot(ds["ctd_temperature"].sel(N_PROF=prof),
         ds["pressure"].sel(N_PROF=prof), c="tab:orange")

ax1.scatter(ds["longitude"].sel(N_PROF=prof),
            ds["latitude"].sel(N_PROF=prof), c="tab:orange")
    
ax2.set_ylim(3500, 2500)
ax2.set_xlim(2.4, 3.25)

fig.tight_layout()
fig.savefig(figure_path / "FigureS1.pdf")

In [ ]:
ds